In [ ]:
######################### DAIKIN #############################
########################## Notes #############################
# The 10 equipments' data don't include 'Bypass Factor' at 
# each row, but include discharge air dry bulb temp.
# Issue to be soloved: The formula requires the apparatus 
# dew point temperature. Use "bypass_adp.ipynb" to calculate.

In [3]:
# !pip3 install metpy
# !pip3 install pathlib # instal if needed

# not work when connect to PNNL VPN

In [15]:
import pandas as pd
import numpy as np
import metpy.calc as mcalc
from metpy.units import units
from pathlib import Path

def loadData(name,nHead,nRows):
    # Load the data from a page in the excel file
    df = pd.read_excel(str(name), sheet_name='Table 1', header=int(nHead), 
                       usecols="C:AC", skiprows=None, nrows=int(nRows), engine="openpyxl")
    # buffer dataframe
    column_names = ["OADB","IDB","IWB","MBH","S2T","kW","airflow"]
    dx = pd.DataFrame(columns=column_names)
    
    return df, dx



In [21]:
def processData1(df,dx):
    ## MBH, S2T, kW
    # ss-drc7
    rows_to_select = [0,1,3,2] # index of rows to select
    names_to_select = ['MBH','S2T','kW','dT']
    
    for i in range(len(rows_to_select)):
        df_temp = df[rows_to_select[i]::7] # internals
        list_2d = []
        for col in df_temp.columns[3:]:
            list_2d.append(df_temp[col].to_list())
        list_1d = []
        for sublist in list_2d:
            for item in sublist:
                list_1d.append(item)   
        dx[names_to_select[i]] = list_1d

    ## Define the 4-element array to repeat
    # IDB, airflow, iwb
    idb_unique = np.unique(df.iloc[:,0].values)
    airflow_unique = np.unique(df.iloc[:,1].values)
    idb_array = np.repeat(idb_unique, len(airflow_unique))
    airflow_array = np.tile(airflow_unique, len(idb_unique))
    iwb_unique = df.columns[3:7]
    iwb_array = np.repeat(iwb_unique, len(airflow_unique)*len(idb_unique))
    # OADB
    oat_unique = np.array([65,75,85,95,105,115])
    oat_array = np.repeat(oat_unique, 24)

    ## Calculate how many times the array has to be repeated and create the repeated array
    idb_repeats = len(dx) // len(idb_array) + (len(dx) % len(idb_array) > 0)
    idb_repeated_array = np.tile(idb_array, idb_repeats)
    airflow_repeated_array = np.tile(airflow_array, idb_repeats)
    iwb_repeats = len(dx) // len(iwb_array) + (len(dx) % len(iwb_array) > 0)
    iwb_repeated_array = np.tile(iwb_array, iwb_repeats)
    oat_repeated_array = np.tile(oat_array, 1)

    ## Slice the repeated array to match the DataFrame's length and assign it to a column
    dx['IDB'] = idb_repeated_array[:len(dx)]
    dx['airflow'] = airflow_repeated_array[:len(dx)]
    dx['IWB'] = iwb_repeated_array[:len(dx)]
    dx['OADB'] = oat_repeated_array[:len(dx)]
    ## replace "-" with NA
    dx.replace('-', np.nan, inplace=True)

    # Sensible MBH
    dx['MBH'] = dx['MBH']*1000
    dx['Sensible MBH'] = dx['MBH']*dx['S2T']

    # The relative_humidity_wet_psychrometric function requires temperature and wet bulb temperature in Kelvin,
    # And pressure in Pascals, we assume standard pressure at sea level: 101325 Pa
    pressure = 101325 * units.pascal
    # Convert Celsius to Kelvin and compute relative humidity
    dx['RH'] = dx.apply(lambda row: mcalc.relative_humidity_wet_psychrometric(pressure,row['IDB'] * units.degF, row['IWB'] * units.degF).to('percent').magnitude, axis=1)
    
    # outlet T
    dx['Out DB'] = dx['IDB']-dx['dT']
    
    return dx



In [32]:
def processData2(df,dx):
    ## MBH, S2T, kW
    # 
    rows_to_select = [0,1,5,2] # index of rows to select
    names_to_select = ['Capacity','S/T','TotalPower','Evap dT']
    
    for i in range(len(rows_to_select)):
        df_temp = df[rows_to_select[i]::6] # internals
        list_2d = []
        for col in df_temp.columns[3:]:
            list_2d.append(df_temp[col].to_list())
        list_1d = []
        for sublist in list_2d:
            for item in sublist:
                list_1d.append(item)   
        dx[names_to_select[i]] = list_1d

    ## Define the 4-element array to repeat
    # IDB, airflow, iwb
    idb_unique = np.unique(df.iloc[:,0].values)
    airflow_unique = np.unique(df.iloc[:,1].values)
    idb_array = np.repeat(idb_unique, len(airflow_unique))
    airflow_array = np.tile(airflow_unique, len(idb_unique))
    iwb_unique = df.columns[3:7]
    iwb_array = np.repeat(iwb_unique, len(airflow_unique)*len(idb_unique))
    # OADB
    oat_unique = np.array([65,75,85,95,105,115])
    oat_array = np.repeat(oat_unique, 24)

    ## Calculate how many times the array has to be repeated and create the repeated array
    idb_repeats = len(dx) // len(idb_array) + (len(dx) % len(idb_array) > 0)
    idb_repeated_array = np.tile(idb_array, idb_repeats)
    airflow_repeated_array = np.tile(airflow_array, idb_repeats)
    iwb_repeats = len(dx) // len(iwb_array) + (len(dx) % len(iwb_array) > 0)
    iwb_repeated_array = np.tile(iwb_array, iwb_repeats)
    oat_repeated_array = np.tile(oat_array, 1)

    ## Slice the repeated array to match the DataFrame's length and assign it to a column
    dx['IDB'] = idb_repeated_array[:len(dx)]
    dx['airflow'] = airflow_repeated_array[:len(dx)]
    dx['IWB'] = iwb_repeated_array[:len(dx)]
    dx['OADB'] = oat_repeated_array[:len(dx)]
    ## replace "-" with NA
    dx.replace('-', np.nan, inplace=True)
    dx.replace('--', np.nan, inplace=True)

    # Sensible MBH
    dx['Sensible Capacity'] = dx['Capacity']*dx['S/T']

    # The relative_humidity_wet_psychrometric function requires temperature and wet bulb temperature in Kelvin,
    # And pressure in Pascals, we assume standard pressure at sea level: 101325 Pa
    pressure = 101325 * units.pascal
    # Convert Celsius to Kelvin and compute relative humidity
    dx['RH'] = dx.apply(lambda row: mcalc.relative_humidity_wet_psychrometric(pressure,row['IDB'] * units.degF, row['IWB'] * units.degF).to('percent').magnitude, axis=1)
    
    # outlet T
    dx['Out DB'] = pd.to_numeric(dx['IDB'])-pd.to_numeric(dx['Evap dT'])
    
    return dx


In [33]:
# headList = [4,46,90,132,176,218]
head = 4
df,dx = loadData('./excel/ss-dfc15.xlsx',head,36)
new = processData2(df,dx)
print(new)

     OADB  IDB  IWB  MBH  S2T  kW  airflow  Capacity   S/T  TotalPower  \
0      65   70   59  NaN  NaN NaN     4500  175388.0  0.59     10925.0   
1      65   70   59  NaN  NaN NaN     5010  177290.0  0.63     10984.0   
2      65   70   59  NaN  NaN NaN     6000  181904.0  0.67     11078.0   
3      65   75   59  NaN  NaN NaN     4500  175490.0  0.71     10916.0   
4      65   75   59  NaN  NaN NaN     5010  177392.0  0.75     10975.0   
..    ...  ...  ...  ...  ...  ..      ...       ...   ...         ...   
139   115   70   71  NaN  NaN NaN     5010       NaN   NaN         NaN   
140   115   70   71  NaN  NaN NaN     6000       NaN   NaN         NaN   
141   115   75   71  NaN  NaN NaN     4500  158797.0  0.49     19876.0   
142   115   75   71  NaN  NaN NaN     5010  160699.0  0.54     19935.0   
143   115   75   71  NaN  NaN NaN     6000  165313.0  0.58     20029.0   

     Evap dT  Sensible Capacity          RH  Out DB  
0      20.37          103478.92   52.721762   49.63  
1  

In [24]:
# ss-drc7
headList = [4,54,102,152,200,250,298,348]
column_names = ["OADB","IDB","IWB","MBH","S2T","kW","airflow"]
buffer = pd.DataFrame(columns=column_names)
for head in headList:
    df,dx = loadData('./excel/ss-drc7.xlsx',head,42)
    new = processData1(df,dx)
    buffer = pd.concat([buffer,new])
    
    ## add on 10/06
    # not drop NA because of the supply fan power
    # buffer = buffer.dropna(subset=['dT']) # drop NA
    
    # rename and order
    order = ["OADB","IDB","IWB","RH", "MBH","Sensible MBH","kW","airflow","Out DB"]
    buffer = buffer[order]
    
filepath = Path('./output/ss-drc7.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)  
buffer.to_csv(filepath)

In [35]:
# ss-dfc15
headList = [4,46,90,132,176,218]
column_names = ["OADB","IDB","IWB","MBH","S2T","kW","airflow"]
buffer = pd.DataFrame(columns=column_names)
for head in headList:
    df,dx = loadData('./excel/ss-dfc15.xlsx',head,36)
    new = processData2(df,dx)
    buffer = pd.concat([buffer,new])
    
    ## add on 10/06
    # rename and order
    order = ["OADB","IDB","IWB","RH", "Capacity","Sensible Capacity","TotalPower","airflow","Out DB"]
    buffer = buffer[order]
    
filepath = Path('./output/ss-dfc15.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)  
buffer.to_csv(filepath)

In [37]:
# ss-dfc5-d
headList = [4,46,90,132,176,218,262,304]
column_names = ["OADB","IDB","IWB","MBH","S2T","kW","airflow"]
buffer = pd.DataFrame(columns=column_names)
for head in headList:
    df,dx = loadData('./excel/ss-dfc5-d.xlsx',head,36)
    new = processData2(df,dx)
    buffer = pd.concat([buffer,new])
    
    ## add on 10/06
    # rename and order
    order = ["OADB","IDB","IWB","RH", "Capacity","Sensible Capacity","TotalPower","airflow","Out DB"]
    buffer = buffer[order]
    
filepath = Path('./output/ss-dfc5-d.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)  
buffer.to_csv(filepath)